# MTCNN + Model Mask Detection

This notebook is run after my neural network model has been trained and the best weights were saved. We will now be able to put together a function that will take pictures fed into the pipeline and determine whether people are wearing masks or not! Due to the large use of RAM, I ran this notebook on Google Colab for better functionality.

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from mtcnn.mtcnn import MTCNN
import cv2
import numpy as np
from pathlib import Path

%pylab inline

Only ran this on Google Colab since it needed to reinstall MTCNN everytime I closed out and logged back into the session.

In [ ]:
!pip install mtcnn

Logged into my Google Drive to import my best saved model and its associated weights.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
model = load_model('input your model here')
model.load_weights('input your model weights here')

## Drawing Boundary Boxes and Labels Function

All credits of this below function goes out to Jason Brownlee on Machine Learning Mastery: https://machinelearningmastery.com/how-to-perform-face-detection-with-classical-and-deep-learning-methods-in-python-with-keras/. The custom text input portion was written by me in order to use my model to predict the label to assign to each face within the image.

In [ ]:
def draw_image_with_boxes(filename, result_list):
    # load the image
    data = cv2.cvtColor(cv2.imread(str(filename)), cv2.COLOR_BGR2RGB)
    # plot the image
    #figure(figsize = (8,8))
    plt.imshow(data)
    # get the context for drawing boxes
    ax = plt.gca()
    # plot each box
    for result in result_list:
        # get coordinates
        x, y, width, height = result['box']
        # create the shape
        rect = Rectangle((x, y), width, height, fill=False, color='blue')
        # draw the box
        ax.add_patch(rect)
        
        # custom text input
        face = data[y: y+height, x: x+width]
        image = cv2.resize(face, (224, 224))
        input_arr = img_to_array(image)
        input_arr = np.array([input_arr]) 
        pred = model.predict_classes(input_arr)[0]
        if pred == 1:
          plt.text((x+(width/3.5)),(y-5), 'Mask', color = 'yellow', size = 'medium')
        else:
          plt.text((x+(width/3.5)),(y-5), 'Face', color = 'red', size = 'medium')
        
    # show the plot
    plt.show()

## Face Detector Function

In [ ]:
def face_detector(path_string):
    # creates a MTCNN object to detect faces
    detector = MTCNN(min_face_size = 35)
    
    # creates a list of the directory items
    picture_path = Path(path_string)
    full_list = list(picture_path.iterdir())
    
    # creates a for loop that will apply a boundary box and label for each original image within the directory
    for each in full_list:
        pixels = plt.imread(each)

        faces = detector.detect_faces(pixels)

        draw_image_with_boxes(each, faces)

## Running the Face Detector Function

In [ ]:
path_string = Path('input the path name of your picture directory here')

In [ ]:
face_detector(path_string)